In [11]:
import pandas as pd
import html
import xml.etree.ElementTree as ET

In [33]:
df_error = pd.read_excel('../do_not_commit/Events/ApplicationEvents.xlsx', sheet_name='ApplicationError')
df_hang = pd.read_excel('../do_not_commit/Events/ApplicationEvents.xlsx', sheet_name='ApplicationHang')
df = pd.concat([df_error, df_hang]).reset_index()

In [102]:
def xml_to_col(xml_str, provider_name):

    # Extract CDATA section
    data_string = xml_str
    unescaped_string = html.unescape(data_string)

    # Extract CDATA section
    start = unescaped_string.find('<![CDATA[') + len('<![CDATA[')
    end = unescaped_string.find(']]>')
    cdata = unescaped_string[start:end]

    # Wrap the CDATA content with a root element
    wrapped_cdata = '<root>' + cdata + '</root>'
    wrapped_cdata

    # Parse the wrapped CDATA as XML
    root = ET.fromstring(wrapped_cdata)
    root[0][-1].text

    if provider_name == 'error':
        row_dict = {
            'FaultingApplicationName':'',
            'AppVersion': '',
            'AppTimestamp': '',
            'FaultingModuleName': '',
            'ModuleVersion': '',
            'ModuleTimestamp': '',
            'ExceptionCode': '',
            'FaultOffset': '',
            'FaultingProcessId': '',
            'FaultingApplicationStartTime': '',
            'FaultingApplicationPath': '',
            'FaultingApplicationModulePath': '',
            'ReportId': '',
            'FaultingPackageFullName': '',
            'FaultingPackageRelativeApplicationID': '',
            'FullPath': '',
            'ProductVersion': '',
            'ProductName': '',
            'Publisher': '',
            'ProgramId': '',
            'FileId': '',
            'FileVersion': ''
        }
    elif provider_name == 'hang':
        row_dict = {
            'Program':'',
            'ProgramVersion': '',
            'ProcessID': '',
            'StartTime': '',
            'TerminationType': '',
            'ApplicationPath': '',
            'ReportID': '',
            'FaultingPackageFullName': '',
            'FaultingPackageRelativeApplicationID': '',
            'HangType': '',
            'BinaryValue': '',
            'FullPath': '',
            'ProductVersion': '',
            'ProductName': '',
            'Publisher': '',
            'ProgramId': '',
            'FileId': '',
            'FileVersion': '',
        }
    elif provider_name == 'windows':
        row_dict = {
            "Data_1": '',
            "FaultBucketType": '',
            "EventName": '',
            "Response": '',
            "CabID": '',
            "ProblemSignatureP1_Application": '',
            "ProblemSignatureP2_AppVersion": '',
            "ProblemSignatureP3": '',
            "ProblemSignatureP4_Module": '',
            "ProblemSignatureP5_ModuleVersion": '',
            "ProblemSignatureP6": '',
            "ProblemSignatureP7_ExceptionCode": '',
            "ProblemSignatureP8": '',
            "ProblemSignatureP9": '',
            "ProblemSignatureP10": '',
            "AttachedFiles": '',
            "AttachedFilesPath": '',
            "AnalysisSymbol": '',
            "RecheckingForSolution": '',
            "ReportID": '',
            "HashedBucket": '',
            "CabGuid": ''
        }
    elif provider_name == 'boot':
        row_dict = {
            "SystemPowerOffTime": '',
            "MachineName": '',
            "AppVersion": '',
            "Reason": '',
            "ExceptionCode": '',
            "ActionDescription": '',
            "SystemPath": '',
            "FullPath": '',
            "ProductVersion": '',
            "ProductName": '',
            "Publisher": '',
            "ProgramId": '',
            "FileId": '',
            "FileVersion": '',
        }

    if provider_name in ['error', 'hang']:
        for i in range(len(root[0])):
            row_dict[list(row_dict.items())[i][0]] = root[0][i].text
        for i in range(len(root[1])):
            if i < 7:
                row_dict[list(row_dict.items())[int(i + len(root[0]))][0]] = root[1][i].text
    elif provider_name in ['windows', 'boot']:
        for i in range(len(root[0])):
            row_dict[list(row_dict.items())[i][0]] = root[0][i].text
    return row_dict

In [ ]:
# RJ 07/17/2023: There are 3 extra attributes in this particular XML:
# FaultBucket, WatsonEventName, UserImpactVector.
data_string = df.loc[98, 'EventDataXML']
unescaped_string = html.unescape(data_string)

# Extract CDATA section
start = unescaped_string.find('<![CDATA[') + len('<![CDATA[')
end = unescaped_string.find(']]>')
cdata = unescaped_string[start:end]

# Wrap the CDATA content with a root element
wrapped_cdata = '<root>' + cdata + '</root>'
wrapped_cdata

# Parse the wrapped CDATA as XML
root = ET.fromstring(wrapped_cdata)
wrapped_cdata